In [1]:
import re
import random
import numpy as np
import pandas as pd
from collections import Counter  

In [2]:
def random_pick_1(seq,probabilities):
    # x = random.uniform(0, 1)#首先随机生成一个0，1之间的随机数
    x = random.randint(1, sum(probabilities))
    cumulative_probability = 0
    for item, item_probability in zip(seq,probabilities):#seq代表待输入的字符串，prob代表各自字符串对应的概率
        cumulative_probability += item_probability#只有当累加的概率比刚才随机生成的随机数大时候，才跳出，并输出此时对应的字符串
        if x <= cumulative_probability:
            break
    return item  

In [3]:
def read_proteome_uniprot():
    '''
    Read the sequences of the proteins in the human proteome. 
    Sequence data are stored in a fasta file     
    Args:
        1. path: The input file containing sequence data of the proteome
            downloaded from the ensemble biomart FTP:
            ftp://ftp.ensembl.org/pub/release-90/fasta/homo_sapiens/.
    Return values:
        1. proteome: A dictionary whose keys are protein ensembl IDs
                    and values are protein sequences
    '''   
    path = "/home/jjia1/viralepitope/uniprot_proteins.csv"
    reference_df = pd.read_csv(path, index_col=0)
    reference_df1 = reference_df.set_index(['accession'])['seq'].to_dict()

    return reference_df1

In [4]:
uniprot = read_proteome_uniprot()

In [5]:
    iedb_csv = "/home/jjia1/viralepitope/viralepitope/iedb_data.txt"
    iedb_df = pd.read_csv(iedb_csv, sep='\t', skiprows=0, low_memory=False, dtype=object)
    iedb_df = np.array(iedb_df)
    
    all_positive_peptide = list(set([p[0] for p in iedb_df]))

In [6]:
    data_dict = {}
    for i in range(len(iedb_df)):
        allele = iedb_df[i][1]        
        if allele not in data_dict.keys():
            data_dict[allele] = [iedb_df[i].tolist()]
        else:
            data_dict[allele].append(iedb_df[i].tolist())

In [7]:
allele

'HLA-A*02:01'

In [8]:
len(iedb_df)

88050

In [9]:
all_neg = []
for allele in data_dict.keys():
    # allele = 'HLA-B*41:06'
    traing_data = data_dict[allele]
    all_length = [len(traing_data[j][0]) for j in range(len(traing_data))]      
    all_length_times = Counter(all_length)

    all_probabilities = []
    for kmer in [8,9,10,11,12,13,14,15]:
        try:              
            probabilities = all_length_times[kmer]
        except:
            probabilities = 0   
        
        all_probabilities.append(probabilities)

In [10]:
all_length

[10, 10, 10, 9, 9, 10, 9, 9, 9, 10, 9]

In [14]:
def Random_peptides():
    '''
    Randomly sample peptides from the proteome
    Args:
        1. proteome: A dictionary of the human proteome.
        Output of the function read_proteome
    Return values:
        1. peptides: Sampled peptides.
    '''
    proteome = read_proteome_uniprot()
    
    iedb_csv = "/home/jjia1/viralepitope/viralepitope/iedb_data.txt"
    iedb_df = pd.read_csv(iedb_csv, sep='\t', skiprows=0, low_memory=False, dtype=object)
    iedb_df = np.array(iedb_df)
    
    all_positive_peptide = list(set([p[0] for p in iedb_df]))
    
    data_dict = {}
    for i in range(len(iedb_df)):
        allele = iedb_df[i][1]        
        if allele not in data_dict.keys():
            data_dict[allele] = [iedb_df[i].tolist()]
        else:
            data_dict[allele].append(iedb_df[i].tolist())

    #randomly generate peptides from the proteome    
    all_neg = []
    for allele in data_dict.keys():
        # allele = 'HLA-B*41:06'
        traing_data = data_dict[allele]
        all_length = [len(traing_data[j][0]) for j in range(len(traing_data))]      
        all_length_times = Counter(all_length)

        all_probabilities = []
        for kmer in [8,9,10,11,12,13,14,15]:
            try:              
                probabilities = all_length_times[kmer]
            except:
                probabilities = 0   
            
            all_probabilities.append(probabilities)
    
        pep_seq = []
        while len(pep_seq) < 10*len(traing_data):  #Set the number of random selections, here is 10 times the number of negative samples as positive samples
            length = random_pick_1([8,9,10,11,12,13,14,15],all_probabilities)  
            accession = random.choice(list(proteome.keys()))
            protein = proteome[accession]
            if len(protein) < length:
                    continue
            pep_start = random.randint(0, len(protein) - length)
            pep = protein[pep_start:pep_start + length]
            
            if set(list(pep)).difference(list('ACDEFGHIKLMNPQRSTVWY')):
                print('No official peptide')
                continue       
                
            if pep in all_positive_peptide:
                print('In positive peptide')
                continue
                
            if pep not in pep_seq:
                pep_seq.append([accession, pep])
    
        for k in pep_seq:
            #k[0] is uniprot ID
            #allele is MHC allele
            #k[1] is ligand sequence
            #all_neg.append([allele, k[0], k[1]])
            all_neg.append([k[1], allele])
            
    return all_neg

In [15]:
neg = Random_peptides()

No official peptide
No official peptide
No official peptide
No official peptide
No official peptide
No official peptide
No official peptide
In positive peptide
No official peptide
No official peptide
No official peptide
No official peptide
No official peptide
No official peptide
In positive peptide
No official peptide
No official peptide
In positive peptide
No official peptide
In positive peptide
No official peptide
In positive peptide
No official peptide
No official peptide
In positive peptide
In positive peptide
No official peptide
No official peptide
In positive peptide
In positive peptide
No official peptide
No official peptide
No official peptide
No official peptide
No official peptide
No official peptide
No official peptide
In positive peptide
No official peptide
No official peptide
No official peptide
No official peptide
No official peptide
No official peptide
No official peptide
No official peptide
No official peptide
No official peptide
No official peptide
In positive peptide


In [16]:
len(neg)

880500

In [17]:
type(neg)

list

In [18]:
neg

[['VLMNHKVFE', 'HLA-A2'],
 ['DLQHLSREER', 'HLA-A2'],
 ['CGFDVLYPP', 'HLA-A2'],
 ['SINNDLNLR', 'HLA-A2'],
 ['DKDTASNEN', 'HLA-A2'],
 ['LLLTMFMGMP', 'HLA-A2'],
 ['KSSTYEELD', 'HLA-A2'],
 ['DCSSASQKD', 'HLA-A2'],
 ['EAYKHRVED', 'HLA-A2'],
 ['PEARVPAQS', 'HLA-A2'],
 ['AYQRLCSYP', 'HLA-A2'],
 ['LVKEGLRGF', 'HLA-A2'],
 ['EGAGTVPPP', 'HLA-A2'],
 ['YNEKQGDVQ', 'HLA-A2'],
 ['AQLSRNSSDT', 'HLA-A2'],
 ['GSMEEAVIL', 'HLA-A2'],
 ['VPGHEGRAG', 'HLA-A2'],
 ['GMYNYLHCD', 'HLA-A2'],
 ['RLSVKENKGLF', 'HLA-A2'],
 ['KNAMLIKGG', 'HLA-A2'],
 ['EVKVLLDQL', 'HLA-A2'],
 ['QSILRFGRH', 'HLA-A2'],
 ['DQDVWILPQA', 'HLA-A2'],
 ['AQGEDLGLM', 'HLA-A2'],
 ['MISSTSVYA', 'HLA-A2'],
 ['PMYYRGAQA', 'HLA-A2'],
 ['AVTPLLEDA', 'HLA-A2'],
 ['LFRKDPNAEG', 'HLA-A2'],
 ['EEEPPTDNQ', 'HLA-A2'],
 ['VRAAGCTRH', 'HLA-A2'],
 ['SNELPVNSH', 'HLA-A2'],
 ['GKPHLGTEKQ', 'HLA-A2'],
 ['LSGFRADIE', 'HLA-A2'],
 ['FLYGSFYYSYM', 'HLA-A2'],
 ['SNSGVGQAV', 'HLA-A2'],
 ['LQTLILPQH', 'HLA-A2'],
 ['QHLQATGVS', 'HLA-A2'],
 ['YCVVCGDKA', 'HLA-A2'],
 [

In [36]:
neg_affinity = pd.DataFrame(neg)

In [37]:
#neg_affinity[3] = 0

In [38]:
neg_affinity.rename(columns = {0:'epitope', 1:'allele'}, inplace = True)

In [39]:
neg_affinity

,epitope,allele
0,VLMNHKVFE,HLA-A2
1,DLQHLSREER,HLA-A2
2,CGFDVLYPP,HLA-A2
3,SINNDLNLR,HLA-A2
4,DKDTASNEN,HLA-A2
...,...,...
880495,AMHKAMLMA,HLA-A*68:12
880496,KRIFSFLDLF,HLA-A*68:12
880497,TSPCRSQVL,HLA-A*68:12
880498,VQSGNLALA,HLA-A*68:12


In [ ]:
len(neg_affinity['allele'].unique())

In [44]:
len(neg_affinity['allele'].unique())

163

In [40]:
#c = ['epitope', 'allele', 'allele.2', 'affinity']
#neg_affinity_reordered = neg_affinity[c]

In [41]:
#neg_affinity_reordered

In [42]:
neg_affinity.to_csv('/home/jjia1/viralepitope/viralepitope/negative_epitope_seq.txt', sep = '\t', header = False, index = False)

In [40]:
#ofile = open("/home/jjia1/viralepitope/viralepitope/negative_epitope_seq.txt", "w")
#for line in neg:
#    ofile.write(line[2]+'\t'+line[0]+'\t'+line[1]+'\n')
#    i = i+1
#ofile.close() 